In [1]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

# Merging speech data with MPs data


In [2]:
speech_df = pd.read_csv('hansard_in_full.csv')
speech_df = speech_df.drop('Unnamed: 0', axis=1)

In [3]:
people_df = pd.read_csv('people.csv')
people_df = people_df.drop('Unnamed: 0', axis=1)

In [4]:
df = speech_df.merge(people_df, how='left', on='person_id')

In [5]:
display(df)

,name,speech_id,person_id,text,speech_date,first_name,family_name,memberships,constituencies,parties
0,Colonel F. B. MILDMAY (addressing himself to t...,uk.org.publicwhip/debate/1919-02-04a.3.1,uk.org.publicwhip/person/15282,"Sir Courtenay Ilbert, in accordance with the G...",1919-02-04,Francis,Mildmay,"['uk.org.publicwhip/member/7070', 'uk.org.publ...","{('1885-11-24', '1886-06-26'): 'Totnes', ('188...",{}
1,Sir HENRY DALZIEL,uk.org.publicwhip/debate/1919-02-04a.5.0,unknown,"Sir Courtenay Ilbert, I am fully conscious tha...",1919-02-04,NaN,NaN,NaN,NaN,NaN
2,Mr. LOWTHER (who was received with general che...,uk.org.publicwhip/debate/1919-02-04a.8.0,unknown,"Sir Courtenay Ilbert,—In accordance with the a...",1919-02-04,NaN,NaN,NaN,NaN,NaN
3,Mr. SPEAKER-ELECT (standing on the upper step ...,uk.org.publicwhip/debate/1919-02-04a.11.0,unknown,"Before taking the Chair, once more as Speaker ...",1919-02-04,NaN,NaN,NaN,NaN,NaN
4,Mr. BONAR LAW (Leader of the House),uk.org.publicwhip/debate/1919-02-04a.11.3,uk.org.publicwhip/person/20693,"Mr. Speaker-Elect,—In the unavoidable absence ...",1919-02-04,Bonar,Law,"['uk.org.publicwhip/member/26103', 'uk.org.pub...","{('1900-10-01', '1906-01-08'): 'Glasgow Blackf...","{('1900-10-01', '1923-10-30'): 'Conservative'}"
...,...,...,...,...,...,...,...,...,...,...
5848598,Nickie Aiken,uk.org.publicwhip/debate/2024-01-11a.551.0,uk.org.publicwhip/person/25817,It was a pleasure to allow that intervention.,2024-01-11,Nickie,Aiken,['uk.org.publicwhip/member/42205'],{},{}
5848599,Nigel Evans,uk.org.publicwhip/debate/2024-01-11a.551.1,uk.org.publicwhip/person/10190,"I have been an MP for 31 years, and it was not...",2024-01-11,Nigel,Evans,"['uk.org.publicwhip/member/2682', 'uk.org.publ...","{('1992-04-09', '1997-04-08'): 'Ribble Valley'...","{('1997-05-01', '2001-05-14'): 'Conservative',..."
5848600,Elliot Colburn,uk.org.publicwhip/debate/2024-01-11a.552.2,uk.org.publicwhip/person/25907,"I thank the Speaker’s Office and you, Mr Deput...",2024-01-11,Elliot,Colburn,['uk.org.publicwhip/member/42633'],{},{}
5848601,Bim Afolami,uk.org.publicwhip/debate/2024-01-11a.554.0,uk.org.publicwhip/person/25661,I congratulate my hon. Friend the Member for C...,2024-01-11,Bim,Afolami,"['uk.org.publicwhip/member/41725', 'uk.org.pub...","{('2017-06-09', '2019-11-06'): 'Hitchin and Ha...","{('2017-06-09', '2019-11-06'): 'Conservative'}"


In [6]:
def extend_current_speech_variables(row, threshold='2019-01-01'):
    # Import datetime to get current day
    from datetime import datetime
    current_day = datetime.strftime(datetime.now(), format="%Y-%m-%d")

    # Check constituencies isn't empty for this row
    if row.constituencies not in [None, []] and not pd.isna(row.constituencies):
        if isinstance(row.constituencies, str):
            constituencies = eval(row.constituencies)
        else:
            constituencies = row.constituencies
        if constituencies.keys() != {}.keys():  # Check that dict is not empty
            # Find the latest date for an entry in the constituency dict
            latest_constituency_date = max(
                [end_date for (start_date, end_date), constituency in constituencies.items()])
            # If latest date is after threshold, then extend entry to now
            if latest_constituency_date >= threshold:
                old_key = [key for key in eval(row.constituencies).keys(
                ) if key[1] == latest_constituency_date][0]
                new_key = (old_key[0], current_day)
                constituencies[new_key] = constituencies.pop(old_key)
    else:
        constituencies = None

    # Repeat for parties
    if row.parties not in [None, []] and not pd.isna(row.parties):
        if isinstance(row.parties, str):
            parties = eval(row.parties)
        else:
            parties = row.parties
        if parties.keys() != {}.keys():
            latest_party_date = max(
                [end_date for (start_date, end_date), party in parties.items()])
            if latest_party_date >= threshold:
                old_key = [key for key in eval(
                    row.parties).keys() if key[1] == latest_party_date][0]
                new_key = (old_key[0], current_day)
                parties[new_key] = parties.pop(old_key)
    else:
        parties = None

    return constituencies, parties


constituencies_parties = df.progress_apply(
    extend_current_speech_variables, axis=1).to_list()
df['constituencies'] = [constituency[0]
                        for constituency in constituencies_parties]
df['parties'] = [party[1] for party in constituencies_parties]

display(df)

100%|██████████| 5848603/5848603 [10:02<00:00, 9713.51it/s] 


,name,speech_id,person_id,text,speech_date,first_name,family_name,memberships,constituencies,parties
0,Colonel F. B. MILDMAY (addressing himself to t...,uk.org.publicwhip/debate/1919-02-04a.3.1,uk.org.publicwhip/person/15282,"Sir Courtenay Ilbert, in accordance with the G...",1919-02-04,Francis,Mildmay,"['uk.org.publicwhip/member/7070', 'uk.org.publ...","{('1885-11-24', '1886-06-26'): 'Totnes', ('188...",{}
1,Sir HENRY DALZIEL,uk.org.publicwhip/debate/1919-02-04a.5.0,unknown,"Sir Courtenay Ilbert, I am fully conscious tha...",1919-02-04,NaN,NaN,NaN,None,None
2,Mr. LOWTHER (who was received with general che...,uk.org.publicwhip/debate/1919-02-04a.8.0,unknown,"Sir Courtenay Ilbert,—In accordance with the a...",1919-02-04,NaN,NaN,NaN,None,None
3,Mr. SPEAKER-ELECT (standing on the upper step ...,uk.org.publicwhip/debate/1919-02-04a.11.0,unknown,"Before taking the Chair, once more as Speaker ...",1919-02-04,NaN,NaN,NaN,None,None
4,Mr. BONAR LAW (Leader of the House),uk.org.publicwhip/debate/1919-02-04a.11.3,uk.org.publicwhip/person/20693,"Mr. Speaker-Elect,—In the unavoidable absence ...",1919-02-04,Bonar,Law,"['uk.org.publicwhip/member/26103', 'uk.org.pub...","{('1900-10-01', '1906-01-08'): 'Glasgow Blackf...","{('1900-10-01', '1923-10-30'): 'Conservative'}"
...,...,...,...,...,...,...,...,...,...,...
5848598,Nickie Aiken,uk.org.publicwhip/debate/2024-01-11a.551.0,uk.org.publicwhip/person/25817,It was a pleasure to allow that intervention.,2024-01-11,Nickie,Aiken,['uk.org.publicwhip/member/42205'],{},{}
5848599,Nigel Evans,uk.org.publicwhip/debate/2024-01-11a.551.1,uk.org.publicwhip/person/10190,"I have been an MP for 31 years, and it was not...",2024-01-11,Nigel,Evans,"['uk.org.publicwhip/member/2682', 'uk.org.publ...","{('1992-04-09', '1997-04-08'): 'Ribble Valley'...","{('1997-05-01', '2001-05-14'): 'Conservative',..."
5848600,Elliot Colburn,uk.org.publicwhip/debate/2024-01-11a.552.2,uk.org.publicwhip/person/25907,"I thank the Speaker’s Office and you, Mr Deput...",2024-01-11,Elliot,Colburn,['uk.org.publicwhip/member/42633'],{},{}
5848601,Bim Afolami,uk.org.publicwhip/debate/2024-01-11a.554.0,uk.org.publicwhip/person/25661,I congratulate my hon. Friend the Member for C...,2024-01-11,Bim,Afolami,"['uk.org.publicwhip/member/41725', 'uk.org.pub...","{('2017-06-09', '2024-01-14'): 'Hitchin and Ha...","{('2017-06-09', '2024-01-14'): 'Conservative'}"


In [7]:
# Using the dictionaries for parties and constituencies, we can find a 'speech_party' and 'speech_constituency'
# These are the party/constituency of the member at the time of giving that particular speech

def get_speech_variables(row):
    # Identify speech date
    speech_date = row.speech_date

    # First check if constituencies dict is empty - if so, speech_constituency is None
    if pd.isna(row.constituencies):
        speech_constituency = None
    else:  # Then, ensure constituencies dict is a dict, not a string
        if isinstance(row.constituencies, str):
            constituencies = eval(row.constituencies)
        else:
            constituencies = row.constituencies
        # Finally, find the constituency entry in the dict where the speech date falls between the start and end dates - this is the speech constituency
        speech_constituency = [constituency for (start_date, end_date), constituency in constituencies.items(
        ) if speech_date >= start_date and speech_date <= end_date]

    # Repeat for parties
    if pd.isna(row.parties):
        speech_party = None
    else:
        if isinstance(row.parties, str):
            parties = eval(row.parties)
        else:
            parties = row.parties
        speech_party = [party for (start_date, end_date), party in parties.items(
        ) if speech_date >= start_date and speech_date <= end_date]

    if speech_constituency not in [None, []]:
        speech_constituency = speech_constituency[0]
    else:
        speech_constituency = None

    if speech_party not in [None, []]:
        speech_party = speech_party[0]
    else:
        speech_party = None

    return speech_constituency, speech_party


speech_constituencies_parties = df.progress_apply(
    get_speech_variables, axis=1).to_list()
df['speech_constituency'] = [constituency[0]
                             for constituency in speech_constituencies_parties]
df['speech_party'] = [party[1] for party in speech_constituencies_parties]

display(df)

100%|██████████| 5848603/5848603 [04:04<00:00, 23969.57it/s]


,name,speech_id,person_id,text,speech_date,first_name,family_name,memberships,constituencies,parties,speech_constituency,speech_party
0,Colonel F. B. MILDMAY (addressing himself to t...,uk.org.publicwhip/debate/1919-02-04a.3.1,uk.org.publicwhip/person/15282,"Sir Courtenay Ilbert, in accordance with the G...",1919-02-04,Francis,Mildmay,"['uk.org.publicwhip/member/7070', 'uk.org.publ...","{('1885-11-24', '1886-06-26'): 'Totnes', ('188...",{},Totnes,None
1,Sir HENRY DALZIEL,uk.org.publicwhip/debate/1919-02-04a.5.0,unknown,"Sir Courtenay Ilbert, I am fully conscious tha...",1919-02-04,NaN,NaN,NaN,None,None,None,None
2,Mr. LOWTHER (who was received with general che...,uk.org.publicwhip/debate/1919-02-04a.8.0,unknown,"Sir Courtenay Ilbert,—In accordance with the a...",1919-02-04,NaN,NaN,NaN,None,None,None,None
3,Mr. SPEAKER-ELECT (standing on the upper step ...,uk.org.publicwhip/debate/1919-02-04a.11.0,unknown,"Before taking the Chair, once more as Speaker ...",1919-02-04,NaN,NaN,NaN,None,None,None,None
4,Mr. BONAR LAW (Leader of the House),uk.org.publicwhip/debate/1919-02-04a.11.3,uk.org.publicwhip/person/20693,"Mr. Speaker-Elect,—In the unavoidable absence ...",1919-02-04,Bonar,Law,"['uk.org.publicwhip/member/26103', 'uk.org.pub...","{('1900-10-01', '1906-01-08'): 'Glasgow Blackf...","{('1900-10-01', '1923-10-30'): 'Conservative'}",Glasgow Central,Conservative
...,...,...,...,...,...,...,...,...,...,...,...,...
5848598,Nickie Aiken,uk.org.publicwhip/debate/2024-01-11a.551.0,uk.org.publicwhip/person/25817,It was a pleasure to allow that intervention.,2024-01-11,Nickie,Aiken,['uk.org.publicwhip/member/42205'],{},{},None,None
5848599,Nigel Evans,uk.org.publicwhip/debate/2024-01-11a.551.1,uk.org.publicwhip/person/10190,"I have been an MP for 31 years, and it was not...",2024-01-11,Nigel,Evans,"['uk.org.publicwhip/member/2682', 'uk.org.publ...","{('1992-04-09', '1997-04-08'): 'Ribble Valley'...","{('1997-05-01', '2001-05-14'): 'Conservative',...",Ribble Valley,Conservative
5848600,Elliot Colburn,uk.org.publicwhip/debate/2024-01-11a.552.2,uk.org.publicwhip/person/25907,"I thank the Speaker’s Office and you, Mr Deput...",2024-01-11,Elliot,Colburn,['uk.org.publicwhip/member/42633'],{},{},None,None
5848601,Bim Afolami,uk.org.publicwhip/debate/2024-01-11a.554.0,uk.org.publicwhip/person/25661,I congratulate my hon. Friend the Member for C...,2024-01-11,Bim,Afolami,"['uk.org.publicwhip/member/41725', 'uk.org.pub...","{('2017-06-09', '2024-01-14'): 'Hitchin and Ha...","{('2017-06-09', '2024-01-14'): 'Conservative'}",Hitchin and Harpenden,Conservative


In [8]:
df.to_csv('hansard_with_mp_details.csv')